In [ ]:
import os

if os.getcwd().endswith('sqlalchemy_utils'):
    os.chdir('..')

from sqlalchemy_utils.models_sqlalchemy_orm import User, Base, Parcellation, Parcel, VoxelwiseArray, ConnectivityArray, ROIArray, GroupLevelMapArray, Domain, Subdomain, Symptom, Synonym, MeshTerm, ResearchPaper, Subject, Connectome, ConnectivityFile, ROIFile, GroupLevelMapFile, StatisticType
from sqlalchemy_utils import db_utils as db_utils
from sqlalchemy.orm import sessionmaker
from sqlalchemy import MetaData, create_engine
import networkx as nx

def detect_circular_references(engine):
    meta = MetaData()
    meta.reflect(bind=engine)
    graph = nx.DiGraph()

    # Build the graph from the foreign key relationships
    for table in meta.tables.values():
        for fk in table.foreign_keys:
            graph.add_edge(fk.column.table.name, fk.parent.table.name, fk=fk)

    try:
        cycle = nx.find_cycle(graph, orientation='original')
        print("Circular reference detected:")
        for edge in cycle:
            from_table = edge[0]
            to_table = edge[1]
            orientation = edge[2]
            fk = graph.edges[(from_table, to_table)]['fk']
            print(f"{from_table} -> {to_table} via foreign key {fk.parent.table.name}.{fk.parent.name} (orientation: {orientation})")
    except nx.exception.NetworkXNoCycle:
        print("No circular references found.")

"""Great, we got it to work-- my new quetstion, is what if there are already models defined in the database?
Am I removing them by running this cell of code?
"""
# Create an SQLAlchemy session, create tables, and check for circular references
engine = create_engine('sqlite:////Users/jt041/repos/lesionbank_new/db.sqlite3')

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Check for circular references
detect_circular_references(engine)
session.close()

In [ ]:
os.getcwd()

db_utils.insert_causes_from_json('sqlalchemy_utils/data/causes.json', session, override_existing=True)
db_utils.insert_domains_from_json('sqlalchemy_utils/data/domains.json', session)
db_utils.insert_symptoms_from_json('sqlalchemy_utils/data/symptoms.json', session)
db_utils.insert_sexes_from_json('sqlalchemy_utils/data/sexes.json', session, override_existing=True)
db_utils.insert_handedness_from_json('sqlalchemy_utils/data/handedness.json', session)
db_utils.insert_connectomes_from_json('sqlalchemy_utils/data/connectomes.json', session)
db_utils.insert_statistic_types_from_json('sqlalchemy_utils/data/statistic_types.json', session)
db_utils.insert_dimensions_from_json('sqlalchemy_utils/data/dimensions.json', session)
db_utils.insert_modalities_from_json('sqlalchemy_utils/data/imaging_modalities.json', session)
db_utils.process_parcellation(db_utils.fetch_atlas_3209c91v(), session)

session.close()

In [ ]:
from sqlalchemy import text, inspect

# result = session.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
# tables = result.fetchall()
# for table in tables:
#     print(table[0])

# inspector = inspect(engine)
# columns = inspector.get_columns('accounts_customuser')
# column_names = [column['name'] for column in columns]
# print("Column names in 'accounts_customuser' table:", column_names)

# for row in session.query(StatisticType).all():
#     print(row.name)
#     print(row.description)

# session.close()

# for row in session.query(Subject).all():
#     print(row.id, row.age, row.connectivity_files)

# for row in session.query(ConnectivityFile).all():
#     print(row.id, row.subject_id)

# for row in session.query(Parcellation).all():
#     print(row.id, row.name)

# for row in session.query(ConnectivityArray).all():
#     print(row.id, row.connectivity_file_id, row.value)

    
# result = session.execute(text("SELECT * FROM test_uploads"))
# for row in result:
#     print(row)

# session.close()